# Philadelphia Building Demolitions 2007-2022
#### MUSA 550 | Myron Bañez

#### Utilizing building demolition data from OpenDataPhilly, this analysis examines different methods to visualize building demolitions in Philadelphia from January 1st 2007 to September 22nd 2022. 

#### Data source: https://www.opendataphilly.org/dataset/building-demolitions/resource/a9068361-0420-44a3-9547-2363081a17bc
#### _____________________________________________________________

## Importing Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import altair as alt
import seaborn as sns
import geopandas as gpd

In [ ]:
censusgeo = gpd.read_file('./ct')
censusgeo

data = './demolition.csv'
demolition = pd.read_csv(data)
demolition

In [ ]:
censusgeo.rename(columns = {'NAME10': 'Tract'}, inplace = True)

## Data Manipulation

### Renaming and dropping columns from the census tracts data.

In [ ]:
censusgeo.rename(columns = {'NAME10': 'Tract'}, inplace = True)
censusgeo

philly = censusgeo.drop(['OBJECTID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'LOGRECNO'], axis=1)

philly = philly.to_crs(epsg=3857)

philly['Tract'] = philly['Tract'].astype(float)

philly.dtypes

### Dropping columns from the building demolition data. 

In [ ]:
demolition_df = demolition.drop(['the_geom', 'the_geom_webmercator', 'objectid', 'addressobjectid', 
                 'parcel_id_num', 'opa_account_num', 'unit_type', 'unit_num', 'opa_owner', 
                 'caseorpermitnumber', 'record_type', 'typeofworkdescription', 'contractorstate', 
                 'contractorzip', 'systemofrecord', 'geocode_x', 'geocode_y', 'council_district', 
                 'posse_jobid', 'applicanttype', 'applicantname', 'contractorname', 'contractortype', 
                 'contractoraddress1', 'contractoraddress2', 'contractorcity'], axis=1)

demolition_df

### The data is melted from wide to tidy format and another column, "status", is dropped from the dataframe.

In [ ]:
demolition_df_clean = pd.melt(
    demolition_df, 
    id_vars=["censustract", "mostrecentinsp", "lat", "lng"],
    value_vars=demolition_df.columns[7],
    value_name="demolished", 
    var_name="status"
)

demolition_df_clean

demolition_df_clean1 = demolition_df_clean.drop(['status'], axis=1)
demolition_df_clean1

### The "mostrecentinsp" column is converted to an appropriate date format to extract just the year.

In [ ]:
demolition_df_clean1["mostrecentinsp"] = pd.to_datetime(demolition_df_clean1["mostrecentinsp"])
date_strings = demolition_df_clean1["mostrecentinsp"].dt.strftime("%Y")

date_strings.iloc[0]
demolition_df_clean1["mostrecentinsp"] = date_strings
demolition_df_clean1

In [ ]:
demolition_df_clean1["mostrecentinsp"] = pd.to_datetime(demolition_df_clean1["mostrecentinsp"])
demolition_df_clean1['Year'] = demolition_df_clean1['mostrecentinsp'].dt.year
demolition_df_clean1 = demolition_df_clean1.drop(['mostrecentinsp'], axis=1)
demolition_df_clean1

In [ ]:
demolition_df_clean1 = demolition_df_clean1.dropna(axis=0)
demolition_df_clean1['Year']= demolition_df_clean1['Year'].astype(int)
demolition_df_clean1

### In order to examine just the completed building demolitions, I will query for just the observations where demolition is complete.

In [ ]:
completed = ["COMPLETED"]
demolition_df_clean1 = demolition_df_clean1.query("demolished in @completed")
demolition_df_clean1

## Analysis

### The sum of demolitions is taken grouped by census tract and year. This is followed by renaming of columns for a more suitable title and dropping an additional column.

In [ ]:
demolition_df_analysis = demolition_df_clean1.groupby(['censustract', 'Year', 'demolished']).size()
demolition_df_analysis = demolition_df_analysis.reset_index()
demolition_df_analysis

demolition_df_analysis.rename(columns = {0:'sum', 'censustract':'Tract'}, inplace = True)
demolition_df_analysis1 = demolition_df_analysis.drop(['demolished'], axis=1)
demolition_df_analysis1

### I am first interested in the total number of demolitions by year, so I am preparing another dataframe to gather this data.

In [ ]:
demolition_df_analysis2 = demolition_df_analysis1.groupby(['Year'])['sum'].sum()
demolition_df_analysis2 = demolition_df_analysis2.reset_index()
demolition_df_analysis2

### Matplotlib is used to visualize this data to show a clear and simple overview of high level information regarding building demolition. As a result of showing total demolitions by year, a bar plot easily depicts this data to compare the years side by side and identify trends.

In [ ]:
### Matplotlib Chart 

ax = demolition_df_analysis2.plot.bar(x='Year', y='sum', rot=0, color='#ffc300')
plt.xticks(rotation = 45)
plt.title("Total Building Demolitions in Philadelphia by Year", fontsize=10)
plt.show()

#### The data shows that 2017 has seen the greatest numver of building demolitions totaling slightly above 1,000. Another point of interest of this chart shows a drastic increase in demolitions from 2008 to 2009, as well as a notable decrease in 2020 which may be attributed to COVID-19.

### I am also interested in what census tract had the greatest number of demolitions. To approach this I will group the initial demolition_df_analysis1 dataframe by tract and retreieve the sum of demolitions.

In [ ]:
phl_demolition_stats = demolition_df_analysis1.groupby(['Tract'])['sum'].sum()
phl_demolition_stats = phl_demolition_stats.reset_index()
phl_demolition_stats

### The transformed dataframe can now be merged to the Philadelphia census tracts.

In [ ]:
phl_demolition = philly.merge(phl_demolition_stats, on='Tract')
phl_demolition

### Visualizing building demolitions by census tracts on a map allows viewers to see if there are any spatial relationships with demolitions to determine if there are particular areas that are more likely to experience demolitions.

In [ ]:
### Altair Chart 1

demolitions_map = phl_demolition.to_crs(epsg=4326)

chart = (
    alt.Chart(demolitions_map)
    .mark_geoshape(stroke="white")
    .encode(
        tooltip=["sum:Q", "Tract:N"],
        color=alt.Color("sum:Q", scale=alt.Scale(scheme='goldred'))
    ).properties(width=700, height=700, title='Total Building Demolitions from 2007-2022 by Census Tract'
)
)

chart

#### From 2007 to 2022, census tract 169.02 has seen the greatest number of building demolitions at 254. The plot shows that a majority of demolitions have occurred in the census tracts just north of Center City.

### Seeing the census tracts that have experienced the greatest number of building demolitions, I now want to see the census tract with the highest amount of demolitions by year, and will return those tract with the greatest sum by year.

In [ ]:
demolition_analysis = demolition_df_analysis1.loc[demolition_df_analysis1.groupby(['Year'])['sum'].idxmax()]
demolition_analysis

### Next, I can gather the data from these tracts from every year to understand if there are trends in building demolition.

In [ ]:
maintracts = [140.00,111.00, 167.01, 151.02, 169.01, 168.00,174.00,169.02,151.02,138.00,
              167.01,138.00,13.00, 149.00,169.02,138.00]
demolition_data = demolition_df_analysis1.query("Tract in @maintracts")

demolition_data

### By converting the data frame into a pivot table and converting the type to float, I use the seaborn package to create an annotated heatmap. This style was chosen in order to clearly input the tract and year in order to understand either trends by year or trends by tract while clearly pointing to the highest total by year. However, some cells are blank, denoting no demolitions in that tract for that year.

In [ ]:
### Seaborn Chart

demo_data_pivot = demolition_data.pivot_table(index='Tract', columns='Year', values='sum', 
               aggfunc='sum')


demolition_data_pivot = demo_data_pivot.astype(float)

plt.figure(figsize=(12, 7))
plt.title("2007-2022 Trends in Census Tracts With The Most Demolitions", fontsize =11)
ax = sns.heatmap(demolition_data_pivot, annot=True, fmt="", cmap="flare", linewidths=.5)
plt.show()

#### Looking at just the top census tracts, census tract 169.01 in 2011 sticks out with the largest amount of building demolitions at 68. Additionally of the top census tracts, 2010 sticks out as a year with many building demolitions simply by the amount and consistency of the darker color of the cells. 

### Another way to visualize this data is by creating a brush plot that allows for interactivity to see building demolitions over time in the individual census tracts.

In [ ]:
### Altair Chart 2: Brush

brush = alt.selection_interval()


alt.Chart(demolition_data).mark_point().encode(
    x=alt.X("Year:N", scale=alt.Scale(zero=False)), 
    y=alt.Y("sum", scale=alt.Scale(zero=False)), 
    color=alt.condition(brush, "sum", alt.value("lightgray"), scale=alt.Scale(scheme='goldred')), 
    tooltip=["Tract", "Year", "sum"], 
).properties(
    width=200, height=200, selection=brush, 
).facet(facet="Tract", columns=4)

#### The plot shows the extreme increase in demolitions in tract 169.01 in 2011. The plot also visualizes consistent and comparitively more demolitions in tracts 167.01, 168, and 169.02 than the other tracts. 

### After further analyzing individual census tracts, I now want to see what share of demolitions the top census tracts had by year using a stacked bar plot. This method allows for color coding visualizing the census tracts within each year while comparing across years and an interactive tool to exactly see the exact number of demolitions.

In [ ]:
### Altair Chart 3: Transforming
(
    alt.Chart(demolition_data)
    .mark_bar()
    .encode(
        x=alt.X('sum:Q', aggregate='sum'),
        y='Year:N',
        color=alt.Color('Tract:N', scale=alt.Scale(scheme='darkred')),
        tooltip=['Year','Tract', 'sum(sum):Q']
    ).properties(
    height=400,
    width=850,
    title='Total Building Demolitions from 2007-2022 with Shares by Census Tract')
)

#### This plot reaffirms that in 2011, census tract 169.01 saw a major increase in building demolitions when compared to the rest of the years. Although tract 169.01 experienced drastically more demolitions in 2011, year 2010 also sticks out as the year with the greatest building demolitions for the top census tracts.